In [ ]:
from dataclasses import dataclass
from numpy import *
import matplotlib.pyplot as plt
from matplotlib import cm

In [ ]:
@dataclass(frozen=True)
class LBEParams:
    """Хранит исходные данные, необходимые для решателя.
    
    `Re`: число Рейнольдса.
    `nx` и `ny`: размеры сетки.
    и так далее...
    """
    Re: float = int(20 + (12 - 1) * (1000 - 20) / (15 - 1))
    U_LB: float = 0.01 + (12 - 1) * (0.1 - 0.01) / (15 - 1)   
    nx: int = 420
    ny: int = 180
    dx: int = 1
    dy: int = 1
    dt: int = 1
    nu: float = U_LB * (nx / 9) / Re
    omega: float = 1 / (3 * nu + 0.5) 


@dataclass(frozen=True)
class Obstacle:
    """Хранит параметры препятствия."""
    x_c: int = LBEParams.nx / 4
    y_c: int = LBEParams.ny / 2
    r: int = LBEParams.ny / 9


class Visualizer:
    """Отвечает за визуализацию результатов."""

    def __init__(self, name):
        self.name = name

    def plot_data(self, ax, data: ndarray, save_dir: str = None, cmap=cm.gray):
        """Построить график данных `data`.
        `iteration`: текущая итерация моделирования.
        `save_dir`: директория сохранения изображений.
        """
        im = ax.imshow(data, cmap=cmap)
        return im

In [ ]:
class LBESolver:
    
    col1 = array([0, 1, 2])
    col2 = array([3, 4, 5])
    col3 = array([6, 7, 8])
    
    def __init__(self, params: LBEParams):
        
        self.speed_vis = Visualizer(r"Скорость $\|{\bf u}\|$")
        self.press_vis = Visualizer(r"Давление $p$")
        self.dens_vis = Visualizer(r"Плотность $\rho$")
        
        self.v = array([[1, 1], [1, 0], [1, -1], [0, 1], [0, 0], [0, -1], [-1, 1], [-1, 0], [-1, -1]])
        self.a = array([1/36, 1/9, 1/36, 1/9, 4/9, 1/9, 1/36, 1/9, 1/36])
        self.dim_grid = LBEParams.nx, LBEParams.ny
        
        self.rho = 1
        self.p = None
        self.u = None
        
        self.fig = None
        self.ax1, self.ax2, self.ax3 = None, None, None
    
    def where_obstacle(self):
        """Определить, какие узлы решётки принадлежат препятствию.
        """
        obstacle_mask = lambda x, y: (x - Obstacle.x_c)**2 + (y - Obstacle.y_c)**2 < Obstacle.r**2
        return fromfunction(obstacle_mask, self.dim_grid)
    
    def init_vel(self):
        """Инициализировать начальное возмущение скорости газа в узлах решётки."""
        inivel = lambda dim, x, y: (1 - dim) * LBEParams.U_LB * (1 + 1e-4 * sin((2 * pi * y) / (LBEParams.ny - 1)))
        return fromfunction(inivel, (2, *self.dim_grid))
    
    def macroscopic(self, fin):
        rho = sum(fin, axis=0)
        u = zeros((2, *self.dim_grid))
        for i in range(9):
            u[0,:,:] += self.v[i,0] * fin[i,:,:]
            u[1,:,:] += self.v[i,1] * fin[i,:,:]
        u /= rho
        p = rho / 3
        return rho, u, p
    
    def equilibrium(self, rho, u):
        feq = zeros((9, *self.dim_grid))
        for i in range(9):
            vu = (self.v[i,0]*u[0,:,:] + self.v[i,1]*u[1,:,:])
            feq[i,:,:] = rho * self.a[i] * (1 + 3*vu + 4.5*vu**2 - 1.5*(u[0]**2 + u[1]**2))
        return feq
    
    def solve(self, iters=100_000, n_pics=100):
        """Основной метод решателя.
        
        ``iters``: количество итераций расчёта.
        `n_pics`: количество картинок, которое необходимо сохранить
        через равные промежутки времени.
        """
        
        obs = self.where_obstacle()
        vel = self.init_vel()
        fin = self.equilibrium(self.rho, vel)
        
        for time in range(iters):
    
            # правая стенка
            fin[self.col3,self.dim_grid[0]-1,:] = fin[self.col3,self.dim_grid[0]-2,:]
            
            self.rho, self.u, self.p = self.macroscopic(fin)
            
            if time % 1000 == 0:
                print(f'iteration: {time} / {iters}')
                self.plot(time)
            
            # левая стенка
            self.u[:,0,:] = vel[:,0,:] 
            self.rho[0,:] = 1 / (1 - self.u[0,0,:]) * (sum(fin[self.col2,0,:], axis=0) + 2 * sum(fin[self.col3,0,:], axis=0))
            
            # вычисление равновесия
            feq = self.equilibrium(self.rho, self.u)
            fin[[0,1,2],0,:] = feq[[0,1,2],0,:] + fin[[8,7,6],0,:] - feq[[8,7,6],0,:]
            
            # столкновение
            fout = fin - LBEParams.omega * (fin - feq) 
            
            # bounce-back
            for i in range(9): 
                fout[i, obs] = fin[8-i, obs]
            
            # распространение
            for i in range(9):
                fin[i,:,:] = roll(roll(fout[i,:,:], self.v[i,0], axis=0), self.v[i,1], axis=1)
                
    def plot(self, iteration):
        print(f'Итерация: {iteration}')
        plt.clf()
        fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(16,8), dpi=100)
        self.fig, self.ax1, self.ax2, self.ax3 = fig, ax1, ax2, ax3
        im_vel = self.speed_vis.plot_data(ax1, sqrt(self.u[0]**2+self.u[1]**2).transpose())
        im_press = self.press_vis.plot_data(ax2, self.p.transpose())
        im_dens = self.dens_vis.plot_data(ax3, self.rho.transpose())
        self.fig.colorbar(im_vel, ax=self.ax1, label=r"$\|{\bf u}\|$")
        self.fig.colorbar(im_press, ax=self.ax2, label=r"$p$")
        self.fig.colorbar(im_dens, ax=self.ax3, extend='both', label=r"$\rho$")
        plt.show()

### Моделирование

In [ ]:
params = LBEParams() # Инициализация ваших исходных данных
solver = LBESolver(params) # Инициализация решателя
solver.solve()